In [5]:
import typer
import warnings
from pathlib import Path
import json

import spacy
from spacy.tokens import DocBin

Import training data

In [9]:
with open("sample_data/topic_training.json") as infile:
    training_data = json.load(infile)

Convert data into spacy training format

In [10]:
nlp = spacy.load("en_core_web_sm")
db = DocBin()

for item in training_data:
    doc = nlp.make_doc(item["text"])
    doc.cats = item["topic"]
    db.add(doc)

db.to_disk("topic_training.spacy")

Run the following commands in the terminal

```bash
python -m spacy init fill-config base_config.cfg config.cfg
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy
```

Test model

In [11]:
nlp = spacy.load("output/model-best/")

In [22]:
nlp("Serve the chowder in bowls, with the halved eggs on top and more corainder scattered over.").cats

{'title': 0.11102817952632904,
 'blurb': 0.11597983539104462,
 'ingredient': 0.12232184410095215,
 'method': 0.560245156288147,
 'other': 0.09042499214410782}